In [2]:
import pandas as pd
import requests

## Download de beneficiários via requests - para correção de encoding

### Por UF

In [176]:
# Fazendo o download do arquivo
url = 'https://www.ans.gov.br/csv/A11053610_22_1_196.csv'
response = requests.get(url)

# Decodificando o conteúdo para a codificação correta
content = response.content.decode('ISO-8859-1')  # Ajuste conforme necessário

# Salvando o arquivo corrigido
with open('data/raw/beneficiarios_planos_uf_set24.csv', 'w', encoding='utf-8') as f:
    f.write(content)

### Por Município

In [177]:
# Fazendo o download do arquivo
url = 'https://www.ans.gov.br/csv/A15453510_22_1_196.csv'
response = requests.get(url)

# Decodificando o conteúdo para a codificação correta
content = response.content.decode('ISO-8859-1')  # Ajuste conforme necessário

# Salvando o arquivo corrigido
with open('data/raw/beneficiarios_planos_mu_set24.csv', 'w', encoding='utf-8') as f:
    f.write(content)

## Populacao total e atendidos por plano de saúde - UF

In [164]:
beneficiarios_planos_uf = pd.read_csv('data/raw/beneficiarios_planos_uf_set24.csv', sep = ';', skiprows=3)
beneficiarios_planos_uf.rename(columns = {'UF': 'uf',
                                          'Assistência_Médica': 'cobertura_plano_saude'}, inplace=True)

beneficiarios_planos_uf

,uf,cobertura_plano_saude
0,Acre,44876
1,Alagoas,384604
2,Amapá,63317
3,Amazonas,621406
4,Bahia,1716590
5,Ceará,1442864
6,Distrito Federal,973626
7,Espírito Santo,1318341
8,Goiás,1389274
9,Maranhão,525773


In [165]:
beneficiarios_planos_uf.to_csv('data/final/beneficiarios_planos_uf_2024.csv', index = False)

In [166]:
populacao_uf = pd.read_csv('data/raw/estimativas_pop_uf_dez24.csv')
populacao_uf.rename(columns = {'Brasil e Unidade da Federação': 'uf',
                               'Ano': 'populacao'}, inplace=True)

populacao_uf

,uf,populacao
0,Rondônia,1746227
1,Acre,880631
2,Amazonas,4281209
3,Roraima,716793
4,Pará,8664306
5,Amapá,802837
6,Tocantins,1577342
7,Maranhão,7010960
8,Piauí,3375646
9,Ceará,9233656


In [167]:
pop_sus_dependente_uf = (populacao_uf.
                         merge(beneficiarios_planos_uf, how = 'left', on = 'uf').
                         eval("pop_sus_dependente = 1 - cobertura_plano_saude/populacao").
                         assign(percentual=lambda x: x['pop_sus_dependente'].apply(lambda v: f"{v * 100:.2f}%")))

pop_sus_dependente_uf

,uf,populacao,cobertura_plano_saude,pop_sus_dependente,percentual
0,Rondônia,1746227,162543,0.906918,90.69%
1,Acre,880631,44876,0.949041,94.90%
2,Amazonas,4281209,621406,0.854853,85.49%
3,Roraima,716793,30859,0.956949,95.69%
4,Pará,8664306,917061,0.894156,89.42%
5,Amapá,802837,63317,0.921133,92.11%
6,Tocantins,1577342,125571,0.920391,92.04%
7,Maranhão,7010960,525773,0.925007,92.50%
8,Piauí,3375646,413275,0.877572,87.76%
9,Ceará,9233656,1442864,0.843739,84.37%


In [121]:
pop_sus_dependente_uf.to_csv('data/final/populacao_sus_dependente_uf_2024.csv', index=False)

## Municípios

### Beneficiários de planos de saúde - Municípios

In [188]:
beneficiarios_planos_mu = pd.read_csv('data/raw/beneficiarios_planos_mu_set24.csv', sep = ';', skiprows=3)

beneficiarios_planos_mu.rename(columns = {'mu': 'mu',
                                          'Assistência_Médica': 'cobertura_plano_saude'}, inplace=True)

beneficiarios_planos_mu[['cod_ibge','municipio']] = beneficiarios_planos_mu['Município'].str.split(' ', expand=True, n=1)

beneficiarios_planos_mu.rename(columns = {'Assistência_Médica': 'cobertura_plano_saude'}, inplace=True)

beneficiarios_planos_mu = beneficiarios_planos_mu[['cod_ibge', 'municipio', 'cobertura_plano_saude']]

beneficiarios_planos_mu

,cod_ibge,municipio,cobertura_plano_saude
0,110001,Alta Floresta D'Oeste,1050
1,110037,Alto Alegre dos Parecis,121
2,110040,Alto Paraíso,240
3,110034,Alvorada D'Oeste,615
4,110002,Ariquemes,7730
...,...,...,...
5589,421265,Pescaria Brava,476
5590,422000,Balneário Rincão,879
5591,431454,Pinto Bandeira,290
5592,500627,Paraíso Das Águas,83


In [ ]:
beneficiarios_planos_mu.to_csv('data/final/beneficiarios_planos_municipios_2024.csv', index= False)

### População estimada

In [189]:
unidades_da_federacao = {
    'cod_uf': ['12', '27', '13', '16', '29', '23', '53', '32', '52', '21', '31', '50',
               '51', '15', '25', '26', '22', '41', '33', '24', '43', '11', '14', '42', '28', '35', '17'],
    'uf': [
        'Acre', 'Alagoas', 'Amazonas', 'Amapá', 'Bahia', 'Ceará', 'Distrito Federal', 'Espírito Santo',
        'Goiás', 'Maranhão', 'Minas Gerais', 'Mato Grosso do Sul', 'Mato Grosso', 'Pará', 'Paraíba', 
        'Pernambuco', 'Piauí', 'Paraná', 'Rio de Janeiro', 'Rio Grande do Norte', 'Rio Grande do Sul',
        'Rondônia', 'Roraima', 'Santa Catarina', 'Sergipe', 'São Paulo', 'Tocantins'
    ],
    'uf_sigla': [
        'AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB',
        'PE', 'PI', 'PR', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SE', 'SP', 'TO'
    ]
}

unidades_da_federacao = pd.DataFrame(unidades_da_federacao)

unidades_da_federacao

,cod_uf,uf,uf_sigla
0,12,Acre,AC
1,27,Alagoas,AL
2,13,Amazonas,AM
3,16,Amapá,AP
4,29,Bahia,BA
5,23,Ceará,CE
6,53,Distrito Federal,DF
7,32,Espírito Santo,ES
8,52,Goiás,GO
9,21,Maranhão,MA


In [191]:
populacao_mu = pd.read_csv('data/raw/estimativas_populacionais_municipios.csv', sep = ';')
populacao_mu.rename(columns = {'Populacao_residente': 'populacao'}, inplace=True)

populacao_mu[['cod_ibge','municipio']] = populacao_mu['Municipio'].str.split(' ', expand=True, n=1)

populacao_mu['cod_uf'] = populacao_mu['cod_ibge'].str[:2]

populacao_mu = populacao_mu[['cod_uf','cod_ibge','municipio','populacao']]

populacao_mu = populacao_mu.merge(unidades_da_federacao, how='left', on = 'cod_uf')

# populacao_mu = populacao_mu[['uf_sigla','uf','cod_ibge','municipio','populacao']]

populacao_mu

,cod_uf,cod_ibge,municipio,populacao,uf,uf_sigla
0,11,110001,ALTA FLORESTA D'OESTE,22853,Rondônia,RO
1,11,110002,ARIQUEMES,108573,Rondônia,RO
2,11,110003,CABIXI,5690,Rondônia,RO
3,11,110004,CACOAL,97637,Rondônia,RO
4,11,110005,CEREJEIRAS,16975,Rondônia,RO
...,...,...,...,...,...,...
5565,52,522200,VIANOPOLIS,15476,Goiás,GO
5566,52,522205,VICENTINOPOLIS,9077,Goiás,GO
5567,52,522220,VILA BOA,4185,Goiás,GO
5568,52,522230,VILA PROPICIO,5982,Goiás,GO


In [195]:
populacao_mu = populacao_mu.merge(beneficiarios_planos_mu[['cod_ibge','municipio']], how = 'left', on = 'cod_ibge', suffixes=['','_padronizado'])

In [196]:
populacao_mu.to_csv('data/final/populacao_municipal_2024.csv', index= False)

In [162]:
# pop_sus_dependente_mu = (populacao_mu.
#                          merge(beneficiarios_planos_mu, how = 'left', on = 'cod_ibge').
#                          eval("pop_sus_dependente = 1 - cobertura_plano_saude/populacao").
#                          assign(percentual=lambda x: x['pop_sus_dependente'].apply(lambda v: f"{v * 100:.2f}%")))

# # pop_sus_dependente_mu = pop_sus_dependente_mu[['cod_ibge', 'municipio','populacao','cobertura_plano_saude','pop_sus_dependente','percentual']]

# pop_sus_dependente_mu